모델 학습은 오랜 시간이 소요되는 작업이므로 학습 결과를 저장하고 불러와 사용할 수 있어야 한다.  
파이토치의 모델은 **직렬화(Serialize)** 와 **역직렬화(Deserialize)** 를 통해 객체를 저장하고 불러올 수 있다.  
<br>
모델을 저장하려면 파이썬의 **피클(Pickle)** 을 활용해 파이썬 객체 구조를 **바이너리 프로토콜(Binary Protocols)** 로 직렬화한다. 모델에 사용된 텐서나 매개변수를 저장한다.  
모델을 불러오려면 저장된 객체 파일을 역직렬화해 현재 프로세스의 메모리에 업로드한다. 이를 통해 계산된 텐서나 매개변수를 불러올 수 있다.  
<br>
*Pytorch 보안 문제가 강화되면서 모델 전체를 불러오는 것이 아닌 파라미터만 불러오는 것이 기본값이 되도록 `torch.load` 함수가 변경되었다.*

# 1. 모델 상태 불러오기

In [6]:
# 모델 저장(파라미터만 저장)
# torch.save(
#     model.state_dict(),
#     "/content/drive/MyDrive/Colab Notebooks/아이펠/AIFFEL_practice/pytorch/base/model_state_dict.pt"
# )

In [5]:
# 비선형 회귀 모델 불러오기
import torch
from torch import nn

class CustomModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer = nn.Linear(2, 1)

  def forward(self, x):
    x = self.layer(x)
    return x

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = CustomModel().to(device) # 모델 구조를 먼저 생성
model_state_dict = torch.load("/content/drive/MyDrive/Colab Notebooks/아이펠/AIFFEL_practice/pytorch/base/model_state_dict.pt",
                              map_location=device) # 저장된 파라미터를 불러오기
model.load_state_dict(model_state_dict) # 모델에 덮어씌우기

with torch.no_grad():
    model.eval()
    inputs = torch.FloatTensor(
        [
            [1 ** 2, 1],
            [5 ** 2, 5],
            [11 ** 2, 11]
        ]
    ).to(device)
    outputs = model(inputs)
    print(outputs)

tensor([[  1.4610],
        [ 69.2141],
        [357.2556]])


단, 모델 상태만 불러오면 모델 구조를 알 수 없으므로 CustomModel 클래스가 동일하게 구현되어 있어야 한다.

# 2. 체크포인트 저장/불러오기
이전에 했었던 비선형 회귀 함수 코드에 체크포인트를 추가한 코드

## 2-1. 체크포인트 저장

In [7]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader


class CustomDataset(Dataset):
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.x = df.iloc[:, 0].values
        self.y = df.iloc[:, 1].values
        self.length = len(df)

    def __getitem__(self, index):
        x = torch.FloatTensor([self.x[index] ** 2, self.x[index]])
        y = torch.FloatTensor([self.y[index]])
        return x, y

    def __len__(self):
        return self.length

class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Linear(2, 1)

    def forward(self, x):
        x = self.layer(x)
        return x

train_dataset = CustomDataset("/content/drive/MyDrive/Colab Notebooks/아이펠/AIFFEL_practice/pytorch/base/data/non_linear.csv")
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True, drop_last=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.0001)

In [8]:
# 체크포인트를 추가한 모델 학습 단계
checkpoint = 1
for epoch in range(10000):
    cost = 0.0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(train_dataloader)

    if (epoch + 1) % 1000 == 0:
        torch.save(
            {
                "model": "CustomModel",
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "cost": cost,
                "description": f"CustomModel 체크포인트-{checkpoint}",
            },
            f"/content/drive/MyDrive/Colab Notebooks/아이펠/AIFFEL_practice/pytorch/base/models/checkpoint-{checkpoint}.pt",
        )
        checkpoint += 1

In [9]:
with torch.no_grad():
    model.eval()
    inputs = torch.FloatTensor(
        [
            [1 ** 2, 1],
            [5 ** 2, 5],
            [11 ** 2, 11]
        ]
    ).to(device)
    outputs = model(inputs)
    print(outputs)

tensor([[  1.7206],
        [ 69.3812],
        [357.0533]])


## 2-2. 체크포인트 불러오기

In [10]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader


class CustomDataset(Dataset):
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.x = df.iloc[:, 0].values
        self.y = df.iloc[:, 1].values
        self.length = len(df)

    def __getitem__(self, index):
        x = torch.FloatTensor([self.x[index] ** 2, self.x[index]])
        y = torch.FloatTensor([self.y[index]])
        return x, y

    def __len__(self):
        return self.length

class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Linear(2, 1)

    def forward(self, x):
        x = self.layer(x)
        return x

train_dataset = CustomDataset("/content/drive/MyDrive/Colab Notebooks/아이펠/AIFFEL_practice/pytorch/base/data/non_linear.csv")
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True, drop_last=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.0001)

In [12]:
# 체크포인트 불러오기
checkpoint = torch.load("/content/drive/MyDrive/Colab Notebooks/아이펠/AIFFEL_practice/pytorch/base/models/checkpoint-6.pt")
model.load_state_dict(checkpoint["model_state_dict"])
optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
checkpoint_epoch = checkpoint["epoch"]
checkpoint_description = checkpoint["description"]
print(checkpoint_description)

CustomModel 체크포인트-6


In [13]:
for epoch in range(checkpoint_epoch + 1, 10000):
    cost = 0.0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss
        if (epoch + 1) % 1000 == 0:
            print(f"Epoch : {epoch+1:4d}, Model : {list(model.parameters())}, Cost : {cost:.3f}")

Epoch : 7000, Model : [Parameter containing:
tensor([[ 3.1035, -1.7031]], requires_grad=True), Parameter containing:
tensor([0.2677], requires_grad=True)], Cost : 0.101
Epoch : 8000, Model : [Parameter containing:
tensor([[ 3.1032, -1.7032]], requires_grad=True), Parameter containing:
tensor([0.2870], requires_grad=True)], Cost : 0.097
Epoch : 9000, Model : [Parameter containing:
tensor([[ 3.1032, -1.7030]], requires_grad=True), Parameter containing:
tensor([0.3047], requires_grad=True)], Cost : 0.088
Epoch : 10000, Model : [Parameter containing:
tensor([[ 3.1029, -1.7031]], requires_grad=True), Parameter containing:
tensor([0.3208], requires_grad=True)], Cost : 0.096


학습 시간이 오래 걸리는 모델은 특정 epoch 마다 체크포인트를 저장하는 것이 좋다.